In [1]:
import codecs,unidecode,yaml,glob,re
import itertools as it
import pandas as pd

In [2]:
nQuizz=10
nAnnee=23
repQuizz="/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/"
fQuizzoodle=repQuizz+"QuizzAll.csv"
fAliasQDLE=repQuizz+"Quizz-Alias.yaml"
fNotesOriginales=repQuizz+"%d-Quizz-Notes.xlsx"%nAnnee
fNotesFinales=repQuizz+"%d-Quizz-Notes-Finales.xlsx"%nAnnee
fCM=repQuizz+"CM.csv"

In [3]:
'''https://stackoverflow.com/a/35491326/3986209'''
def invert_dict(d): 
    inverse = dict() 
    for key in d: 
        # Go through the list that is saved in the dict:
        for item in d[key]:
            # Check if in the inverted dict the key exists
            if item not in inverse: 
                # If not create a new list
                inverse[item] = [key] 
            else: 
                inverse[item].append(key) 
    return inverse

# Fichier Etudiants

In [4]:
dfCM=pd.read_csv(filepath_or_buffer=fCM,sep="\t",encoding="utf8")
dfCM.set_index(u"Numéro d'identification",inplace=True)
dfCM[u"Code étudiant"]=dfCM.index.astype(unicode)
dfCM

,Nom de famille,Prénom,Adresse de courriel,Dernier téléchargement depuis ce cours,Code étudiant
Numéro d'identification,,,,,
436893,ABELA--FRANZETTI,GABRIEL,gabriel.abela--franzetti@etu.u-bordeaux-montai...,1697705395,436893
440340,ABERARD,FLORELLE,florelle.aberard@etu.u-bordeaux-montaigne.fr,1697705395,440340
435916,ADAM,EULALIE,eulalie.adam@etu.u-bordeaux-montaigne.fr,1697705395,435916
442931,AIT-SIDI-ALI,YASSINE,yassine.ait-sidi-ali@etu.u-bordeaux-montaigne.fr,1697705395,442931
438199,AKPINAR,NESLIHAN,neslihan.akpinar@etu.u-bordeaux-montaigne.fr,1697705395,438199
386382,ALVAREZ,ORIANE,oriane.alvarez@etu.u-bordeaux-montaigne.fr,1697705395,386382
440881,AMOUZOU,RACHEL,rachel.amouzou@etu.u-bordeaux-montaigne.fr,1697705395,440881
441950,ANDERSON,FREDERICK JOHN,frederick-john.anderson@etu.u-bordeaux-montaig...,1697705395,441950
437791,APPRIOU,MAURINE,maurine.appriou@etu.u-bordeaux-montaigne.fr,1697705395,437791


# Fichier Alias

In [5]:
try:
    with codecs.open(fAliasQDLE, 'r', encoding="utf8") as inputfile:
        correspondances=yaml.safe_load(inputfile)
    alias=correspondances["alias2etu"]
except:
    alias={}
alias

{'APPRIOU-Maurine': ['437791'],
 'Aberard Florelle': ['440340'],
 'Aberard_Florelle': ['440340'],
 'Adam-eulalie': ['435916'],
 'Adriana Mangiarratti': ['442806'],
 'Adriana-Mangiarratti': ['442806'],
 'Agathe raynal': ['436720'],
 'Agatheraynal': ['436720'],
 'Akpinar Neslihan': ['438199'],
 'Amelia Collins': ['437856'],
 'Angelina': ['438425'],
 'Angelina_Boukono': ['438425'],
 'Assimay nahi': ['436153'],
 'Audrey-luu': ['435643'],
 'Axelle-Meteau, Anais-Vieira': ['437803', '437028'],
 'BONNEAU MELISSA': ['439214'],
 'BONNEAU Melissa': ['439214'],
 u'BONNEAU-M\xe9lissa,TROUILLET-M': ['439214', '435953'],
 'Benhamouda-souha': ['442845'],
 'Boileau-Valentine': ['440209'],
 'Bordelait Lizzie': ['439788'],
 'Bordelait-Lizzie': ['439788'],
 'Boukang': ['438425'],
 'Boy-eleonore': ['441173'],
 'Camille Baldy': ['436384'],
 'Camille Raynaud': ['438388'],
 'Camille raynaud': ['438388'],
 'Camille-Baldy': ['436384'],
 'Carla Rumeau': ['438098'],
 'Carla.Rum.Alv': ['438098'],
 'Cassandra- CLAV

In [15]:
colCM=[u"id",u"nom",u"prénom",u"email",u"ine"]
# colQuizz=["pseudo","points","temps"]

try:
    dfCM=pd.read_csv(fCM,sep="\t",encoding="utf8",header=0,names=colCM)
    lQuizz=pd.read_csv(fQuizzoodle,sep=";",encoding="utf8",header=0)
    lQuizz.columns=["pseudo"]+lQuizz.columns.tolist()[1:]
    # lQuizz.pseudo=lQuizz.pseudo.str.strip()
except:
    dfCM=None
    lQuizz=None

In [17]:
# dfCM=pd.read_csv(filepath_or_buffer=fCM,sep="\t",encoding="utf8",header=0,names=colCM)
lQuizz

,pseudo,0(2),1(4),2(6),3(9),4(11),5(13),6(16),7(18),8(20),...,20(48),21(51),22(53),23(55),24(58),25(60),26(62),27(65),28(67),29(69)
0,,E,F,G,B,A,D,B,A,A,...,D,B,A,D,B,C,D,D,B,A
1,MÉCHAIN-Hermès,?,?,?,?,?,?,?,?,?,...,C,C,A,D,C,?,?,D,A,?
2,Nora_galissaire,D,F,D,D,A,D,B,A,?,...,?,C,A,D,A,B,?,?,D,?
3,margot-lesieur,?,?,?,?,A,?,D,A,A,...,C,B,D,D,D,C,B,A,B,A
4,siebert-e,?,F,G,D,A,D,D,A,B,...,B,D,A,D,C,?,B,A,D,B
5,Marylou.doy,?,D,G,B,A,D,D,?,B,...,B,?,A,B,D,B,B,A,C,B
6,lena_pavie,E,F,D,D,A,D,B,A,?,...,?,C,D,D,A,B,C,?,D,?
7,Flavie Brouxel,D,F,D,B,A,D,?,B,B,...,C,D,A,?,?,D,D,C,C,C
8,Merlet Tahina,D,F,G,D,A,D,B,?,A,...,C,B,A,?,D,A,D,B,?,?
9,Tsifanavi Lucia,C,E,E,C,A,D,D,D,B,...,B,A,A,B,A,B,A,?,C,A


In [18]:
dfCM["PN"]=dfCM[u"prénom"].str.lower().str.strip()+" "+dfCM[u"nom"].str.lower().str.strip()
dfCM["NP"]=dfCM[u"nom"].str.lower().str.strip()+" "+dfCM[u"prénom"].str.lower().str.strip()

In [19]:
lQuizz

,pseudo,0(2),1(4),2(6),3(9),4(11),5(13),6(16),7(18),8(20),...,20(48),21(51),22(53),23(55),24(58),25(60),26(62),27(65),28(67),29(69)
0,,E,F,G,B,A,D,B,A,A,...,D,B,A,D,B,C,D,D,B,A
1,MÉCHAIN-Hermès,?,?,?,?,?,?,?,?,?,...,C,C,A,D,C,?,?,D,A,?
2,Nora_galissaire,D,F,D,D,A,D,B,A,?,...,?,C,A,D,A,B,?,?,D,?
3,margot-lesieur,?,?,?,?,A,?,D,A,A,...,C,B,D,D,D,C,B,A,B,A
4,siebert-e,?,F,G,D,A,D,D,A,B,...,B,D,A,D,C,?,B,A,D,B
5,Marylou.doy,?,D,G,B,A,D,D,?,B,...,B,?,A,B,D,B,B,A,C,B
6,lena_pavie,E,F,D,D,A,D,B,A,?,...,?,C,D,D,A,B,C,?,D,?
7,Flavie Brouxel,D,F,D,B,A,D,?,B,B,...,C,D,A,?,?,D,D,C,C,C
8,Merlet Tahina,D,F,G,D,A,D,B,?,A,...,C,B,A,?,D,A,D,B,?,?
9,Tsifanavi Lucia,C,E,E,C,A,D,D,D,B,...,B,A,A,B,A,B,A,?,C,A


In [20]:
pseudos=lQuizz[~(lQuizz["pseudo"].isin(alias))]["pseudo"].str.strip().tolist()
print "\n".join(pseudos)


margot-lesieur
siebert-e
Flavie Brouxel
mon-a sal-a
MARCENAT Marine POURIEUX Laura
Mangiarratti Adriana
Lina Reim
Susset
amandine.berloin
Victoria


In [21]:
for nP,pseudo in enumerate(pseudos):
    print pseudo
    sPseudo = raw_input("Nom a chercher:\n")
    if sPseudo:
        print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]
        idPseudo=raw_input("id :\n")
        if idPseudo:
            alias[pseudo]=idPseudo

Nom a chercher:
 


margot-lesieur


Nom a chercher:
 lesieur


                PN      id
89  margot lesieur  441656


id :
 441656


...

id :
 429252


Victoria


Nom a chercher:
 victor


                            PN      id
123  victoria prazmo-cygalidis  442706


id :
 442706


In [22]:
# pseudo=u"Cotentin, Pailhes"
# alias[pseudo]=["414180","411916"]

In [23]:
sPseudo="pailh"
print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]

Empty DataFrame
Columns: [PN, id]
Index: []


In [24]:
pn=dfCM["PN"].tolist()
np=dfCM["NP"].tolist()

In [25]:
for pseudo in pseudos:
    if unidecode.unidecode(pseudo.lower()) in pn:
        print pseudo,dfCM[dfCM["PN"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["PN"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]
    elif unidecode.unidecode(pseudo.lower()) in np:
        print pseudo,dfCM[dfCM["NP"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["NP"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]

Flavie Brouxel 437820
Mangiarratti Adriana 442806


In [ ]:
for al in alias:
    if not isinstance(alias[al],list):
        if " " in alias[al]:
            alias[al]=alias[al].split(" ")
        else:
            alias[al]=[alias[al]]

In [66]:
invAlias=invert_dict(alias)
#invAlias[385732]=[u'adiarra001, cbeauvais004']
correspondances={"alias2etu":alias,"etu2alias":invAlias}
correspondances

{'alias2etu': {'APPRIOU-Maurine': ['437791'],
  'Aberard Florelle': ['440340'],
  'Aberard_Florelle': ['440340'],
  'Adam-eulalie': ['435916'],
  'Adriana Mangiarratti': ['442806'],
  'Adriana-Mangiarratti': ['442806'],
  'Agathe raynal': ['436720'],
  'Agatheraynal': ['436720'],
  'Akpinar Neslihan': ['438199'],
  'Amelia Collins': ['437856'],
  'Angelina': ['438425'],
  'Angelina_Boukono': ['438425'],
  'Assimay nahi': ['436153'],
  'Audrey-luu': ['435643'],
  'Axelle-Meteau, Anais-Vieira': ['437803', '437028'],
  'BONNEAU MELISSA': ['439214'],
  'BONNEAU Melissa': ['439214'],
  u'BONNEAU-M\xe9lissa,TROUILLET-M': ['439214', '435953'],
  'Benhamouda-souha': ['442845'],
  'Boileau-Valentine': ['440209'],
  'Bordelait Lizzie': ['439788'],
  'Bordelait-Lizzie': ['439788'],
  'Boukang': ['438425'],
  'Boy-eleonore': ['441173'],
  'Camille Baldy': ['436384'],
  'Camille Raynaud': ['438388'],
  'Camille raynaud': ['438388'],
  'Camille-Baldy': ['436384'],
  'Carla Rumeau': ['438098'],
  'Ca

In [67]:
with codecs.open(fAliasQDLE, 'w', encoding="utf8") as outfile:
    yaml.safe_dump(correspondances, outfile, default_flow_style=False)

# Bonnes réponses pour les différents quizz du QuizzAll
- sur la deuxième ligne lQuizz[1], il y a toutes les bonnes réponses
- on coupe le Quizz All par groupe de trois questions pour chaque quizz d'origine
 - *quizzReponses*

In [68]:
with codecs.open(fQuizzoodle,"r",encoding="utf8") as input:
    lQuizz=input.readlines()

In [69]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]    

In [70]:
def getReponses(ligne):
    result={}
    ligneEls=[l.strip() for l in ligne.split(";")]
    pseudo=ligneEls[0]
    reponses=ligneEls[1:]
    for num,rep in enumerate(chunks(reponses,3)):
        result["%02d"%(num+2)]=rep
    return pseudo,result    

In [71]:
temp,quizzReponses=getReponses(lQuizz[1])
quizzReponses

{'02': [u'E', u'F', u'G'],
 '03': [u'B', u'A', u'D'],
 '04': [u'B', u'A', u'A'],
 '05': [u'B', u'C', u'D'],
 '06': [u'D', u'D', u'C'],
 '07': [u'B', u'A', u'C'],
 '08': [u'C', u'A', u'D'],
 '09': [u'B', u'A', u'D'],
 '10': [u'B', u'C', u'D'],
 '11': [u'D', u'B', u'A']}

In [72]:
def markReponses(reponses):
    scores={}
    for q in quizzReponses:
#         print q
        score=0
        for n,r in enumerate(quizzReponses[q]):
#             print n,r,reponses[q][n]
            if r==reponses[q][n]:
                score+=1
        scores[q]=int(round(float(score)*20/3,0))    
    return scores

# Notes du pseudo pour les différents quizz
- sur la ligne du pseudo, on a toutes les réponses
- on coupe les réponses par groupe de trois réponses
- on compare avec les bonnes réponses

In [73]:
alias[u"mon-a sal-a"]

['438210', '438330']

In [74]:
quizzNotes={}
for l in lQuizz[2:]:
    nom,reponses=getReponses(l)
#     print nom,reponses
    
    marks=markReponses(reponses)
    
#     print nom, " ; ".join([str(marks[k]) for k in sorted(marks.keys())])
    
    if nom in alias:
#         print nom,alias[nom]
        akas=alias[nom]
    else:
        print "pas de",nom
        akas=[]
        
    for aka in akas:
        if aka in quizzNotes:
            print aka,"déjà dans les notes",invAlias[aka]
            print quizzNotes[aka]
            print marks
            for k,v in quizzNotes[aka].iteritems():
                if v<marks[k]:
                    quizzNotes[aka][k]=marks[k]
            print quizzNotes[aka]
        else:
            quizzNotes[aka]=marks
print 
print "===================="
print
for aka in sorted(quizzNotes.keys()):
#    print chercherIdentifiant(aka)
    print aka, ", ".join([str(quizzNotes[aka][k]) for k in sorted(quizzNotes[aka].keys())])



416084 7, 13, 7, 13, 7, 13, 13, 13, 7, 0
427761 7, 7, 13, 13, 7, 13, 7, 13, 7, 0
427846 13, 13, 13, 7, 7, 7, 7, 20, 0, 0
428240 7, 7, 0, 13, 0, 7, 7, 13, 7, 0
428316 13, 20, 13, 7, 13, 7, 13, 7, 0, 0
429252 13, 20, 13, 7, 13, 7, 13, 7, 0, 0
431717 13, 13, 13, 7, 7, 13, 20, 13, 13, 0
435376 20, 13, 7, 7, 7, 0, 0, 7, 7, 0
435468 13, 0, 7, 0, 20, 7, 7, 7, 0, 0
435514 20, 13, 7, 13, 7, 13, 7, 13, 7, 0
435581 13, 13, 7, 20, 13, 7, 7, 13, 0, 0
435616 7, 13, 7, 20, 7, 13, 7, 13, 13, 0
435641 13, 7, 0, 7, 13, 0, 7, 13, 0, 0
435643 0, 7, 7, 13, 7, 7, 7, 13, 0, 13
435651 7, 13, 7, 20, 7, 7, 13, 13, 0, 0
435774 13, 13, 20, 7, 0, 7, 0, 13, 7, 0
435916 13, 7, 13, 13, 13, 7, 7, 7, 13, 13
435953 7, 13, 7, 13, 13, 7, 7, 20, 0, 13
435982 7, 13, 7, 13, 0, 13, 7, 7, 7, 7
436164 7, 13, 0, 13, 7, 0, 13, 0, 13, 0
436286 7, 7, 20, 13, 7, 7, 0, 13, 13, 7
436330 0, 20, 7, 13, 13, 7, 13, 13, 7, 7
436384 13, 20, 7, 13, 13, 7, 0, 13, 13, 0
436524 13, 13, 0, 0, 20, 0, 7, 20, 13, 0
436528 13, 13, 7, 20, 13, 7, 13

In [75]:
quizzNotes[u"443367"]

{'02': 0,
 '03': 13,
 '04': 0,
 '05': 7,
 '06': 0,
 '07': 0,
 '08': 7,
 '09': 7,
 '10': 0,
 '11': 7}

In [76]:
dfNotesOriginales=pd.read_excel(fNotesOriginales)
dfNotesOriginales[u"Numéro d'identification"]=dfNotesOriginales[u"Numéro d'identification"].astype("str")

In [77]:
# dfCampus=dfNotesOriginales[[u"Numéro d'identification"]+u"Nom Prénom".split(" ")+["Quizz%02d (Brut)"%(i+1) for i in range(1,10)]]
dfCampus=dfNotesOriginales[[u"Numéro d'identification"]+u"Nom Prénom".split(" ")+["Q%02d"%(i+1) for i in range(1,11)]]
dfCampus.set_index(u"Numéro d'identification",inplace=True)
# dfCampus.loc["417001"]

In [78]:
nouvellesNotes={}
for etu in quizzNotes:
    print etu
    if etu in dfCampus.index:
        for q in quizzNotes[etu]:
            # if quizzNotes[etu][q] > dfCampus.loc[etu,"Quizz%s (Brut)"%q] or dfCampus.loc[etu,"Quizz%s (Brut)"%q]=="-":
            if quizzNotes[etu][q] > dfCampus.loc[etu,"Q%s"%q] or dfCampus.loc[etu,"Q%s"%q]=="-":
                # print u"Quizz %s"%q,dfCampus.loc[etu,"Quizz%s (Brut)"%q],"=>",quizzNotes[etu][q]
                print u"Quizz %s"%q,dfCampus.loc[etu,"Q%s"%q],"=>",quizzNotes[etu][q]
                if not etu in nouvellesNotes:
                    nouvellesNotes[etu]={}
                nouvellesNotes[etu][q]=quizzNotes[etu][q]
                # dfCampus.loc[etu,"Quizz%s (Brut)"%q]=quizzNotes[etu][q]
                dfCampus.loc[etu,"Q%s"%q]=quizzNotes[etu][q]
    else:
        print "PB", etu, "pas de notes originales"
        print quizzNotes[etu]
        nouvellesNotes[etu]=quizzNotes[etu]
len(nouvellesNotes)

438388
Quizz 06 11 => 13
Quizz 07 16 => 20
Quizz 08 12 => 20
438496
Quizz 09 - => 7
441133
Quizz 02 16 => 20
Quizz 06 - => 13
Quizz 07

/Users/gilles/opt/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


 - => 13
Quizz 05 - => 13
Quizz 08 - => 13
Quizz 09 - => 7
440209
Quizz 06 - => 7
Quizz 07 - => 7
Quizz 05 - => 7
429252
Quizz 06 11 => 13
Quizz 07 - => 7
Quizz 08 - => 13
Quizz 09 - => 7
Quizz 11 - => 0
Quizz 10 - => 0
437865
Quizz 10 - => 0
435916
Quizz 11 12 => 13
435641
Quizz 06 11 => 13
Quizz 09 - => 13
Quizz 11 - => 0
435643
Quizz 06 - => 7
Quizz 04 - => 7
Quizz 05 - => 13
Quizz 08 - => 7
Quizz 09 - => 13
Quizz 11 12 => 13
Quizz 10 - => 0
440486
Quizz 06 - => 7
Quizz 05 - => 7
440762
Quizz 05 14 => 20
Quizz 09 12 => 13
438729
Quizz 06 - => 13
Quizz 07 - => 7
Quizz 05 - => 7
Quizz 09 - => 13
440840
Quizz 03 13 => 20
Quizz 07 - => 13
Quizz 04 - => 7
Quizz 09 18 => 20
439788
Quizz 06 11 => 13
Quizz 08 - => 0
Quizz 09 15 => 20
436720
Quizz 06 11 => 13
Quizz 07 13 => 20
Quizz 08 13 => 20
438764
Quizz 02 19 => 20
442548
438098
Quizz 02 17 => 20
Quizz 06 10 => 13
441678
Quizz 08 13 => 20
Quizz 11 - => 0
443153
Quizz 02 16 => 20
Quizz 03 17 => 20
Quizz 06 - => 13
Quizz 07 14 => 20
Quizz 

77

In [79]:
dfCampus.loc["441388"]

Nom       LETOURNEUR
Prénom        LOHANE
Q02               20
Q03               14
Q04               13
Q05                7
Q06                0
Q07                7
Q08                0
Q09                0
Q10                0
Q11               12
Name: 441388, dtype: object

In [88]:
numId="440661"
dfCampus.loc[numId],dfNotesOriginales[dfNotesOriginales[u"Numéro d'identification"]==numId]

(Nom       FOUQUET
 Prénom      ZELIG
 Q02            13
 Q03            20
 Q04            13
 Q05            17
 Q06            13
 Q07            16
 Q08            13
 Q09            13
 Q10            10
 Q11            15
 Name: 440661, dtype: object,    Numéro d'identification      Nom Prénom  \
 55                  440661  FOUQUET  ZELIG   
 
                           Adresse de courriel Q02 Q03 Q04 Q05 Q06 Q07 Q08 Q09  \
 55  zelig.fouquet@etu.u-bordeaux-montaigne.fr  11  14  13  17  11  16  13  12   
 
    Q10 Q11  
 55  10  15  )

In [81]:
dfCampus.to_excel(fNotesFinales)